In [ ]:
# Import required libraries
import os
from datetime import datetime, timedelta

# Import the financial_news_rag module
from financial_news_rag.marketaux import MarketauxNewsFetcher
from financial_news_rag.data import fetch_financial_news, search_entities

# Optional: Configure logging to see more details
import logging
logging.basicConfig(level=logging.INFO)

# Marketaux Financial News API Demo

This notebook demonstrates how to use the Financial News RAG module to fetch financial news articles from the Marketaux API.

The examples show different ways to filter and search for news by:
- Company symbols
- Sentiment
- Date ranges
- Search terms
- Entity types

## Basic News Fetching

Let's start by fetching some recent news articles for specific companies:

In [ ]:
# Fetch news for Tesla and Apple
result = fetch_financial_news(
    symbols=['TSLA', 'AAPL'],  # Tesla and Apple
    days_back=7,               # Last 7 days
    language=['en'],           # English only
    filter_entities=True       # Only return relevant entities
)

# Display the number of articles found
print(f"Found {result['meta']['found']} articles, returning {result['meta']['returned']}")

# Display the first article
if result['data']:
    article = result['data'][0]
    print(f"\nTitle: {article['title']}")
    print(f"Source: {article['source']}")
    print(f"Published: {article['published_at']}")
    print(f"URL: {article['url']}")
    print(f"\nSnippet: {article['snippet']}")
    
    # Show entities
    if article.get('entities'):
        print("\nEntities:")
        for entity in article['entities']:
            sentiment = entity.get('sentiment_score', 0)
            sentiment_str = "positive" if sentiment > 0.1 else "negative" if sentiment < -0.1 else "neutral"
            print(f"  - {entity['symbol']}: {entity.get('name', '')} ({sentiment_str}, score: {sentiment})")
else:
    print("No articles found")


## Filtering by Sentiment

We can filter articles by sentiment score to find positive or negative news:

In [ ]:
# Fetch positive news (sentiment >= 0.2)
positive_news = fetch_financial_news(
    symbols=['MSFT', 'AMZN'],   # Microsoft and Amazon
    sentiment_gte=0.2,          # Positive sentiment threshold
    days_back=7,               # Last 7 days
    language=['en']            # English only
)

print(f"Found {positive_news['meta']['found']} positive articles")

# Fetch negative news (sentiment <= -0.2)
negative_news = fetch_financial_news(
    symbols=['MSFT', 'AMZN'],   # Microsoft and Amazon
    sentiment_lte=-0.2,         # Negative sentiment threshold
    days_back=7,               # Last 7 days
    language=['en']            # English only
)

print(f"Found {negative_news['meta']['found']} negative articles")

# Show article headlines with sentiment
def show_articles_with_sentiment(articles, title):
    if not articles['data']:
        print(f"No {title} articles found")
        return
        
    print(f"\n{title} headlines:")
    for i, article in enumerate(articles['data'], 1):
        # Find the overall sentiment for this article by averaging entity sentiments
        sentiments = [e.get('sentiment_score', 0) for e in article.get('entities', [])]
        avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else 0
        sentiment_str = f"({avg_sentiment:.2f})"
        
        print(f"{i}. {article['title']} {sentiment_str}")

show_articles_with_sentiment(positive_news, "Positive")
show_articles_with_sentiment(negative_news, "Negative")

## Filtering by Date Range

We can specify exact date ranges for our news search:

In [ ]:
# Define date range (May 1-7, 2025)
start_date = datetime(2025, 5, 1)
end_date = datetime(2025, 5, 7)

# Fetch news for a specific date range
date_filtered_news = fetch_financial_news(
    symbols=['NVDA'],             # Nvidia
    date_range=(start_date, end_date),
    language=['en'],              # English only
    filter_entities=True          # Only return relevant entities
)

print(f"Found {date_filtered_news['meta']['found']} articles between {start_date.date()} and {end_date.date()}")

# Show headlines with dates
if date_filtered_news['data']:
    print("\nHeadlines:")
    for i, article in enumerate(date_filtered_news['data'], 1):
        # Parse the published date
        published_at = article['published_at']
        if isinstance(published_at, str):
            # Convert ISO format string to datetime
            published_at = published_at.replace('Z', '+00:00')
            published_at = datetime.fromisoformat(published_at)
        
        # Format date for display
        date_str = published_at.strftime("%Y-%m-%d %H:%M")
        print(f"{i}. [{date_str}] {article['title']}")
else:
    print("No articles found in this date range")

## Text Search

We can search for specific terms or phrases in the news articles:

In [ ]:
# Search for specific terms
search_results = fetch_financial_news(
    search="AI earnings",       # Search for AI-related earnings news
    days_back=7,               # Last 7 days
    language=['en'],           # English only
    filter_entities=True       # Only return relevant entities
)

print(f"Found {search_results['meta']['found']} articles matching 'AI earnings'")

# Show search results with relevance scores
if search_results['data']:
    print("\nSearch results:")
    for i, article in enumerate(search_results['data'], 1):
        relevance = article.get('relevance_score', 'N/A')
        print(f"{i}. {article['title']} (Relevance: {relevance})")
        
        # List entities in this article
        entities = article.get('entities', [])
        if entities:
            print("   Entities:")
            for entity in entities:
                print(f"    - {entity.get('symbol', 'N/A')}: {entity.get('name', 'Unknown')}")
        print()  # Add blank line between articles
else:
    print("No articles found matching the search terms")

## Searching for Entities

We can also search for entity information directly:

In [ ]:
# Search for AI-related companies
entity_results = search_entities(
    search="AI",                # Search term
    entity_types=["equity"],    # Only stocks
    countries=["us"]           # US companies only
)

print(f"Found {entity_results['meta']['found']} entities matching 'AI'")

# Display entity information
if entity_results['data']:
    print("\nEntities:")
    for i, entity in enumerate(entity_results['data'], 1):
        print(f"{i}. {entity.get('symbol', 'N/A')}: {entity.get('name', 'Unknown')}")
        print(f"   Type: {entity.get('type', 'N/A')}")
        print(f"   Industry: {entity.get('industry', 'N/A')}")
        print(f"   Country: {entity.get('country', 'N/A')}")
        print()
else:
    print("No entities found matching the search terms")

## Conclusion

In this notebook, we've demonstrated several ways to fetch and filter financial news articles using the Marketaux API integration:

1. Basic news fetching for specific companies
2. Filtering by sentiment (positive/negative)
3. Filtering by date range
4. Text search for specific terms
5. Entity information search

These capabilities can be combined to create powerful financial news applications, sentiment analysis tools, and more.